**Installing necessary Libraries**

In [ ]:
!pip install diffusers transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [1]:
! pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.8 MB/s eta 0:00:00


In [2]:
! pip install npx

**Saving the necessary pretrained models for various applications**

### Next Word Prediction using GPT-2

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

#Save the model and tokenizer
torch.save(tokenizer, "GPT2_tokenizer.pth")
torch.save(model, "GPT2.pth")

### Chatbot

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load pre-trained model and tokenizer with padding set to left
model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')
model = AutoModelForCausalLM.from_pretrained(model_name)

#Save the model and tokenizer
torch.save(tokenizer, "chatbot_tokenizer.pth")
torch.save(model, "chatbot.pth")

### Sentiment Analysis

In [ ]:
from transformers import pipeline
import torch

# Load pre-trained sentiment-analysis pipeline
model = pipeline("sentiment-analysis")

#Save the pipeline
torch.save(model,"sentiment_analysis.pth")

### Summary Generation

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer
import torch

# Load pre-trained model and tokenizer
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

#Save the model and tokenizer
torch.save(tokenizer, "summary_tokenizer.pth")
torch.save(model, "summary.pth")

### Question and Answer Model based on the given Context

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

# Load the pre-trained model and tokenizer
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

#Save the model and tokenizer
torch.save(tokenizer, "QandA_tokenizer.pth")
torch.save(model, "QandA.pth")

### Image Generation

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

# Load the pre-trained Stable Diffusion model
model_id = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

#Save the model
torch.save(pipe,"image_generator.pth")

**Streamlit dashboard**

In [3]:
%%writefile app.py
import torch
import streamlit as st

##================================================/ USER DEFINED FUNCTIONS / ===============================================##
#Predict next word
def predict_next_word(prompt, model, tokenizer, top_k=5):
    model.eval()
    # Tokenize input text
    inputs = tokenizer(prompt, return_tensors='pt')

    # Generate predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get logits of the last token in the sequence
    next_token_logits = outputs.logits[:, -1, :]

    # Filter the top k tokens by their probability
    top_k_tokens = torch.topk(next_token_logits, top_k).indices[0].tolist()

    # Decode the top k tokens to their corresponding words
    predicted_tokens = [tokenizer.decode([token]) for token in top_k_tokens]

    return predicted_tokens


#Generate text
def generate_text(prompt, model, tokenizer, max_length=50):
    generated = tokenizer.encode(prompt, return_tensors='pt')

    model.eval()
    with torch.no_grad():
        for _ in range(max_length):
            outputs = model(generated)
            next_token_logits = outputs.logits[:, -1, :]
            next_token = torch.argmax(next_token_logits, dim=-1)
            generated = torch.cat((generated, next_token.unsqueeze(0)), dim=1)
            if next_token == tokenizer.eos_token_id:
                break

    return tokenizer.decode(generated[0], skip_special_tokens=True)


#Chatbot
def chat_with_model(prompt, chat_history_ids, model, tokenizer):
    model.eval()

    # Encode the new user input, add the eos_token and return a tensor in PyTorch
    new_user_input_ids = tokenizer.encode(prompt + tokenizer.eos_token, return_tensors='pt')

    # Append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if chat_history_ids is not None else new_user_input_ids

    # Generate a response
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # Decode the response and print it
    response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

    return response, chat_history_ids


#Text Summarization
def summarize(text, model, tokenizer, max_length=130, min_length=30, do_sample=False):
    model.eval()
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


#Question Answering
def answer_question(question, context,model,tokenizer):
    # Tokenize input question and context
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")

    # Get input ids and attention mask
    input_ids = inputs["input_ids"].tolist()[0]

    # Get the model's output
    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Get the most likely beginning and end of answer with the argmax of the score
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    # Convert tokens to the answer
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    return answer


#Image generation
def generate_image(prompt, pipe):
    with torch.no_grad():
        image = pipe(prompt).images[0]
    return image


##================================================/ STREAMLIT DASHBOARD / ===============================================##

#Page_configuration
st.set_page_config(
    page_title = 'NLP applications',
    page_icon = '🖥️',
    layout = 'wide')

# st.sidebar.markdown("Hello! This is a Multi-functional NLP and Image Generation App⚛️")
st.sidebar.header(":green[APPS]")
app = st.sidebar.selectbox(label='**Choose an app**',options = ("Next Word Predictor", "Story Completion",
      "Chatbot", "Sentiment Analysis", "Text Summarization", "Question Answering", "Image Generator"),
      index=None, key='app', placeholder="Select application")

if app == None:
  st.header("Hello! This is a Multi-functional NLP and Image Generation App⚛️")
  st.write("Streamlit app ✅ created by Abinaya Ganesh")

#Predict_next_word
if app == "Next Word Predictor":

  st.sidebar.markdown("## :green[GUIDE]")
  st.sidebar.info("**Next Word Predictor app predicts few words that can be the next word for the text that you enter.**")
  st.sidebar.markdown("### :green[EXAMPLE]")
  st.sidebar.write("**Text given** - The quick brown fox was not very friendly")
  st.sidebar.write("**Next word predictions** - to/with/,/./and")

  st.subheader(":green[**NEXT WORD PREDICTOR💭**]")
  prompt = st.text_input("**Enter the text to predict next word**",key='prompt')
  predict = st.button(label = '**Predict**', key='predict')

  if predict:
    #Load model
    model = torch.load("GPT2.pth")
    tokenizer = torch.load("GPT2_tokenizer.pth")

    #Function call
    predicted_words = predict_next_word(prompt, model, tokenizer)
    st.write(f"**The text you entered:** {prompt}")
    st.write(f"**The predicted words are:** {'/'.join(predicted_words)}")

#Story completion
if app == "Story Completion":

  st.sidebar.markdown("## :green[GUIDE]")
  st.sidebar.info("**Story Completion app completes the piece of story that you enter with the maximum number of words being 50**")
  st.sidebar.markdown("### :green[EXAMPLE]")
  st.sidebar.write("**Text given** - The quick brown fox was about six months of")
  st.sidebar.write("**Completed story** - The quick brown fox was about six months of age when he was attacked by a group of wolves.\
  He was taken to the hospital where he was treated for a broken jaw and a broken leg.")

  st.subheader(":green[**STORY COMPLETION APP📖**]")
  text = st.text_area("**Enter the text which you want the app to complete. Word limit - 20**",key='text')
  generate = st.button(label = '**Generate**', key='generate')

  if generate:
    #Load model
    model = torch.load("GPT2.pth")
    tokenizer = torch.load("GPT2_tokenizer.pth")

    #Function call
    generated_text = generate_text(text, model, tokenizer)
    st.write(f"**The completed text is:** \n {generated_text}")

#Chatbot
if app == "Chatbot":

  st.sidebar.markdown("## :green[GUIDE]")
  st.sidebar.info("**This app is a chatbot. Start chatting with the bot!**")

  st.subheader(":green[**CHAT BOT🤖**]")

  #Load model
  tokenizer = torch.load("chatbot_tokenizer.pth")
  model = torch.load("chatbot.pth")

  chat_history_ids = None

  # Initialize chat history
  if "messages" not in st.session_state:
      st.session_state.messages = []

  # Display chat messages from history on app rerun
  for message in st.session_state.messages:
      with st.chat_message(message["role"]):
          st.markdown(message["content"])

  # React to user input
  if prompt := st.chat_input("Say something"):
      # Display user message in chat message container
      st.chat_message("user").markdown(prompt)
      # Add user message to chat history
      st.session_state.messages.append({"role": "user", "content": prompt})

      response, chat_history_ids = chat_with_model(prompt, chat_history_ids, model, tokenizer)
      # Display assistant response in chat message container
      with st.chat_message("assistant"):
          st.write(response)
      # Add assistant response to chat history
      st.session_state.messages.append({"role": "assistant", "content": response})


#Sentiment Analysis
if app == 'Sentiment Analysis':

  st.sidebar.markdown("## :green[GUIDE]")
  st.sidebar.info("**Sentiment Analysis app provides a Positive or Negative anaysis along with a score for the text that you provide**")
  st.sidebar.markdown("### :green[EXAMPLE]")
  st.sidebar.write("**Text given** - The weather is gloomy and I feel sad.")
  st.sidebar.write("**Sentiment** - Negative")

  st.subheader(":green[**SENTIMENT ANALYSIS😀/😔**]")
  sentiment_text = st.text_input("**Enter a sentence for sentiment analysis**",key='sentiment_text')
  analyse = st.button("**Analyse**",key='analyse')

  if analyse:
    #Load model
    model = torch.load("sentiment_analysis.pth")

    results = model(sentiment_text)
    st.write(f"**Text:** {sentiment_text}")
    st.write(f"**Sentiment:** {results[0]['label']}, **Score:** {results[0]['score']:.4f}")


#Text Summarization
if app == "Text Summarization":

  st.sidebar.markdown("## :green[GUIDE]")
  st.sidebar.info("**Text Summarization app provides a summary of the paragraph that you enter. Word limit - 130**")


  st.subheader(":green[**TEXT SUMMARIZATION📝**]")
  para = str(st.text_area("**Enter a paragraph to summarise**",key='para'))
  summary_bt = st.button(label='**Summarize**', key='summarize')

  if summary_bt:
    #Load model
    tokenizer = torch.load("summary_tokenizer.pth")
    model = torch.load("summary.pth")

    #Function call
    summary = summarize(para, model, tokenizer)
    st.write("**Original text:**\n", para)
    st.write("\n**Summary:**\n", summary)


#Question Answering
if app == "Question Answering":

  st.sidebar.markdown("## :green[GUIDE]")
  st.sidebar.info("This app answers the questions that you ask provided the context")
  st.sidebar.markdown("### :green[EXAMPLE]")
  st.sidebar.write("**Context** - The Transformers library by Hugging Face provides general-purpose architectures for Natural Language Understanding (NLU) and Natural Language Generation (NLG) with pretrained models in over 100 languages.\
  These models can be used on a variety of tasks, including text classification, information extraction, question answering, summarization, translation, and text generation.")
  st.sidebar.write("**Question** - What does the Transformers library provide?")
  st.sidebar.write("**Answer** - general - purpose architectures for natural language understanding ( nlu ) and natural language generation ( nlg )")

  st.subheader(":green[**QUESTION ANSWERING❓**]")
  context = st.text_area("**Enter a paragraph that you want to ask questions from**")
  question = st.text_input("**Enter a question**",key='question')
  answer_bt = st.button(label = "**Get Answer**", key='answer_bt')

  if answer_bt:
    #Load model
    tokenizer = torch.load("QandA_tokenizer.pth")
    model = model = torch.load("QandA.pth")

    #Function call
    answer = answer_question(question, context,model,tokenizer)
    st.write(f"**Question:** {question}")
    st.write(f"**Answer:** {answer}")


#Image Generation
if app == "Image Generator":

  st.sidebar.markdown("## :green[GUIDE]")
  st.sidebar.info("This app generates an image for the prompt given.")
  st.sidebar.markdown("### :green[EXAMPLE]")
  st.sidebar.write("**Prompt**: A futuristic cityscape at sunset")

  st.subheader(":green[**IMAGE GENERATOR🖼️**]")
  prompt = st.text_input("**Enter a prompt**",key='prompt')
  image_bt = st.button("**Generate**",key='image')


  if image_bt:
    #Load model
    pipe = torch.load("image_generator.pth")

    #Function call
    image = generate_image(prompt, pipe)

    st.image(image, caption=prompt)


Writing app.py


**Create a local tunnel and run Streamlit app**

In [4]:
!wget -q -O - ipv4.icanhazip.com

35.196.77.43


In [5]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.196.77.43:8501

npx: installed 22 in 8.055s
your url is: https://cold-clocks-doubt.loca.lt
  Stopping...
^C
